In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv("processed_data/final_pre-processed1.csv")[:12805*4]

In [3]:
df = df.drop('Unnamed: 0',axis=1)
df["Ratings"] = df["Ratings"].astype(int)

In [4]:
test_Ratings = df["Ratings"]
rate = []
for i in test_Ratings:
    if int(i) >=3:
        rate.append(1)
    else:
        rate.append(0)
df["Ratings"] = rate

In [5]:
text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(df['reviewText'])
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [6]:
X = text_vectorizer.transform(df['reviewText']).toarray()
y = df['Ratings']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

In [7]:
global_model = Sequential()
global_model.add(Dense(128,input_dim=X_train.shape[1]))
global_model.add(Dense(1,activation='softmax'))

In [8]:
global_model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [9]:
NUM_ROUNDS = 10
NUM_CLIENTS = 10
BATCH_SIZE = 256
EPOCHS_PER_ROUND = 10
LEARNING_RATE = 0.1
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-7

In [10]:
x_train_clients = np.array_split(X_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(X_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [11]:
from sklearn.metrics import classification_report
import keras 

client_models = []
client_losses = []
client_train_losses = []
client_classification = []
client_acc = []
global_maes = []

client_models = []
for i in range(NUM_CLIENTS):
    local_model = Sequential()
    local_model.add(Dense(128,input_dim=X_train.shape[1]))
    local_model.add(Dense(1,activation='softmax'))
    local_model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    # Initialize an empty list to store the training loss
    training_losses = []

    # Define a custom callback to capture training loss
    class LossHistory(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            training_losses.append(logs['loss'])

    #  Fit the model with the training data and capture the training loss
    history_callback = local_model.fit(x_train_clients[i],y_train_clients[i], epochs=EPOCHS_PER_ROUND, callbacks=[LossHistory()])

    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    preds = local_model.predict(x_test_clients[i])
    c2 = classification_report(y_test_clients[i],preds)
    client_models.append(local_model)
    client_train_losses.append(training_losses)
    client_acc.append(acc[1])
    client_classification.append(c2)
    client_models.append(local_model)

Epoch 1/10
129/129 [==============================] - 2s 14ms/step - loss: 0.3063 - accuracy: 0.9353
Epoch 2/10
129/129 [==============================] - 2s 13ms/step - loss: 0.1727 - accuracy: 0.9353
Epoch 3/10
129/129 [==============================] - 2s 13ms/step - loss: 0.1009 - accuracy: 0.9353
Epoch 4/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0553 - accuracy: 0.9353
Epoch 5/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0338 - accuracy: 0.9353
Epoch 6/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0226 - accuracy: 0.9353
Epoch 7/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0164 - accuracy: 0.9353
Epoch 8/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0129 - accuracy: 0.9353
Epoch 9/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0105 - accuracy: 0.9353
Epoch 10/10
33/33 [==============================] - 0s 3ms/step


/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch 1/10
129/129 [==============================] - 2s 12ms/step - loss: 0.3071 - accuracy: 0.9353
Epoch 2/10
129/129 [==============================] - 2s 12ms/step - loss: 0.1700 - accuracy: 0.9353
Epoch 3/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0966 - accuracy: 0.9353
Epoch 4/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0520 - accuracy: 0.9353
Epoch 5/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0313 - accuracy: 0.9353
Epoch 6/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0199 - accuracy: 0.9353
Epoch 7/10
129/129 [==============================] - 1s 12ms/step - loss: 0.0139 - accuracy: 0.9353
Epoch 8/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0101 - accuracy: 0.9353
Epoch 9/10
129/129 [==============================] - 1s 12ms/step - loss: 0.0077 - accuracy: 0.9353
Epoch 10/10
33/33 [==============================] - 0s 3ms/step


/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch 1/10
129/129 [==============================] - 2s 13ms/step - loss: 0.3071 - accuracy: 0.9361
Epoch 2/10
129/129 [==============================] - 2s 13ms/step - loss: 0.1699 - accuracy: 0.9361
Epoch 3/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0982 - accuracy: 0.9361
Epoch 4/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0533 - accuracy: 0.9361
Epoch 5/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0311 - accuracy: 0.9361
Epoch 6/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0196 - accuracy: 0.9361
Epoch 7/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0136 - accuracy: 0.9361
Epoch 8/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0100 - accuracy: 0.9361
Epoch 9/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0076 - accuracy: 0.9361
Epoch 10/10
33/33 [==============================] - 0s 3ms/step


...

/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch 1/10
129/129 [==============================] - 2s 14ms/step - loss: 0.3052 - accuracy: 0.9346
Epoch 2/10
129/129 [==============================] - 2s 13ms/step - loss: 0.1743 - accuracy: 0.9346
Epoch 3/10
129/129 [==============================] - 2s 13ms/step - loss: 0.1038 - accuracy: 0.9346
Epoch 4/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0588 - accuracy: 0.9346
Epoch 5/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0352 - accuracy: 0.9346
Epoch 6/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0227 - accuracy: 0.9346
Epoch 7/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0155 - accuracy: 0.9346
Epoch 8/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0112 - accuracy: 0.9346
Epoch 9/10
129/129 [==============================] - 2s 14ms/step - loss: 0.0084 - accuracy: 0.9346
Epoch 10/10
32/32 [==============================] - 0s 3ms/step


/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Epoch 1/10
129/129 [==============================] - 2s 14ms/step - loss: 0.3036 - accuracy: 0.9351
Epoch 2/10
129/129 [==============================] - 2s 12ms/step - loss: 0.1641 - accuracy: 0.9351
Epoch 3/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0870 - accuracy: 0.9351
Epoch 4/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0463 - accuracy: 0.9351
Epoch 5/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0280 - accuracy: 0.9351
Epoch 6/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0185 - accuracy: 0.9351
Epoch 7/10
129/129 [==============================] - 2s 13ms/step - loss: 0.0136 - accuracy: 0.9351
Epoch 8/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0109 - accuracy: 0.9351
Epoch 9/10
129/129 [==============================] - 2s 12ms/step - loss: 0.0089 - accuracy: 0.9351
Epoch 10/10
32/32 [==============================] - 0s 3ms/step


/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sujaykumar/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [12]:
data = [client_acc, client_train_losses, client_classification]
data = list(map(list, zip(*data)))
column_names = ['Accuracy', 'Train Losses', 'Classification']
df1 = pd.DataFrame(data, columns=column_names)
df1.to_csv('results.csv')

In [13]:
acc = global_model.evaluate(X_test,y_test)
print(acc)

321/321 [==============================] - 2s 7ms/step - loss: 0.6927 - accuracy: 0.9307
[0.692729651927948, 0.930691123008728]


In [14]:
# SERVER ADAM OPTIMIZER 
fedadam = tf.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=BETA_1,beta_2 = BETA_2,epsilon=EPSILON)

In [15]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(NUM_CLIENTS):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [16]:
acc = global_model.evaluate(X_test,y_test)
print(acc)

321/321 [==============================] - 2s 7ms/step - loss: 0.1929 - accuracy: 0.9307
[0.1928771436214447, 0.930691123008728]


In [17]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer=fedadam, loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(X_train,y_train,epochs=5,batch_size=20)

: 

In [ ]:
acc = fed_model.evaluate(X_test,y_test)
print(acc)

220/220 [==============================] - 2s 9ms/step - loss: 0.9681 - accuracy: 0.6972
[0.9680886268615723, 0.6971680521965027]
